<a href="https://colab.research.google.com/github/pabloochocobar/CE888/blob/main/TweetEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
##importing libraries
## data manipulation
import pandas as pd
import numpy as np
import re
import string
 
## methods and stop words text processing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
 
## Machine learning libraries
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")
!pip install transformers

     |████████████████████████████████| 2.1MB 9.5MB/s 
     |████████████████████████████████| 3.3MB 44.5MB/s 
     |████████████████████████████████| 901kB 48.2MB/s 


In [9]:
## creating a stopwords set
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
## loading the Datasets
def load_dataset(filepath, cols):
    """
    reads the CSV file to return a dataframe with specified column names
    """
    df = pd.read_csv(filepath, encoding = 'latin-1')
    df.columns = cols
    return df
    ##Getting rid of the unwanted columns
def delete_redundant_cols(df, cols):
    """
    Deletes unwanted columns(cols) from the dataframe.
    """
    for col in cols:
        del df[col]
    return df

In [6]:
## Preprocessing the Tweet Text
nltk.download('punkt')
def preprocess_tweet(tweet):
    """
    Runs a set of transformational steps to preprocess the text of the tweet.
    """
    # convert all text to lowercase
    tweet = tweet.lower()
    #remove any URLs
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet, flags=re.MULTILINE)
    #remove punctuations
    tweet = tweet.translate(str.maketrans("", "",string.punctuation))
    #remove user@ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#',"",tweet)
    #remove stopwords
    tweet_tokens = word_tokenize(tweet)
    filtered_words =[word for word in tweet_tokens if word not in stop_words]
    #stemming
    ps =PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]
    #lemmatizing
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]
    return " ".join(lemma_words)
nltk.download('punkt')
nltk.download('wordnet')

NameError: ignored

In [12]:
# download label mapping for emotions
import csv
import urllib.request
task= 'emotion'
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [column[1] for column in csvreader if len(column) > 1]
labels

['anger', 'joy', 'optimism', 'sadness']

In [13]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [14]:
# # TF
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [15]:
!wget https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_text.txt

--2021-04-29 09:41:03--  https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_text.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132523 (129K) [text/plain]
Saving to: ‘test_text.txt.1’

test_text.txt.1     100%[===================>] 129.42K  --.-KB/s    in 0.01s   

2021-04-29 09:41:03 (9.68 MB/s) - ‘test_text.txt.1’ saved [132523/132523]



In [4]:
emotion_text="test_text.txt"
etext = open(emotion_text, "r").read().split("\n")
etext

["@user @user what do these '1/2 naked pics' have to do with anything? They're not even like that. ",
 'OH: “I had a blue penis while I was this” [playing with Google Earth VR] ',
 "@user @user That's coming, but I think the victims are going to be Medicaid recipients. ",
 'I think I may be finally in with the in crowd #mannequinchallenge  #grads2014 @user ',
 '@user Wow,first Hugo Chavez and now Fidel Castro. Danny Glover, Michael Moore, Oliver Stone, and Sean Penn are running out of heroes. ',
 'Savchenko now Saakashvili took drug test live on Ukraine TV. To prove they are not drug-fueled loonies? ',
 'How many more days until opening day? 😩 ',
 "Twitter's #ThankYouObama Shows Heartfelt Gratitude To POTUS ",
 'All CSG and Fracking all around Australia is to cease and Mining Entities held Accountable! - Sign the Petition! ',
 '@user @user @user @user @user @user take away illegals and dead people and Trump wins popular vote too. ',
 'An interesting security vulnerability - albeit not 

In [5]:
gold_2 = etext
for k in range(len(etext)):
  text = preprocess_tweet(etext[k])
  encoded_input = tokenizer(text, return_tensors='tf')
  output = model(encoded_input)
  scores = output[0][0].numpy()
  scores = softmax(scores)
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    if i == 0:
        lb =l;  # to get first ranking number
    s = scores[ranking[i]]
    #print(f"{i+1}) {l} {np.round(float(s), 4)}")
    s=len(labels);
  for j in range(len(labels)):
      if lb == labels[j]:
         value = j;
  gold_2[k]=value;
gold_2

NameError: ignored

In [1]:
!wget https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/predictions/emotion.txt

--2021-04-29 10:22:38--  https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/predictions/emotion.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2842 (2.8K) [text/plain]
Saving to: ‘emotion.txt’

emotion.txt         100%[===================>]   2.78K  --.-KB/s    in 0s      

2021-04-29 10:22:39 (51.5 MB/s) - ‘emotion.txt’ saved [2842/2842]



In [2]:
gold_2
for i in range(len(gold_2)):
   if gold_2[i]==0:
      gold_2[i]='0'
   if gold_2[i]==1:
      gold_2[i]='1'
   if gold_2[i]==2:
      gold_2[i]='2'
   if gold_2[i]==3:
      gold_2[i]='3'
gold_2

NameError: ignored

In [ ]:
# Read the emotion.

example1 = "emotion.txt"
pred = open(example1, "r").read().split("\n")
pred =gold_2

In [ ]:
!wget https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_labels.txt

In [ ]:
test_labels = "test_labels.txt"
gold = open(test_labels, "r").read().split("\n")
gold

In [ ]:
from sklearn.metrics import classification_report
result = classification_report(gold,pred,output_dict=True)
tweeteval_result = result['macro avg']['f1-score']
for k in result:
  print(k, result[k])
  print(f"{'-'*30}\nTweetEval Score (emotion): {tweeteval_result}")

In [ ]:
# download label mapping for sentiments
import csv
import urllib.request
task= 'sentiments'
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [column[1] for column in csvreader if len(column) > 1]
labels

In [ ]:
!wget https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_text.txt

In [ ]:
sentiment_text="test_text.txt"
stext = open(sentiment_text, "r").read().split("\n")
stext

In [ ]:
sentiment_text="test_text.txt"
stext = open(sentiment_text, "r").read().split("\n")
stext

In [ ]:
gold_2 = stext;
for k in range(len(stext)):
  text = preprocess_tweet(stext[k])
  encoded_input = tokenizer(text, return_tensors='tf')
  output = model(encoded_input)
  scores = output[0][0].numpy()
  scores = softmax(scores)
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    if i == 0:
        lb =l;  # to get first ranking number
    s = scores[ranking[i]]
    #print(f"{i+1}) {l} {np.round(float(s), 4)}")
    s=len(labels);
  for j in range(len(labels)):
      if lb == labels[j]:
         value = j;
  gold_2[k]=value;
gold_2

In [ ]:
!wget https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/predictions/sentiment.txt

In [ ]:
gold_2
for i in range(len(gold_2)):
   if gold_2[i]==0:
      gold_2[i]='0'
   if gold_2[i]==1:
      gold_2[i]='1'
   if gold_2[i]==2:
      gold_2[i]='2'

In [ ]:
# Read the sentiment.

example1 = "sentiment.txt"
pred = open(example1, "r").read().split("\n")
pred =gold_2

In [ ]:
!wget https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_labels.txt

In [ ]:
test_labels = "test_labels.txt"
gold = open(test_labels, "r").read().split("\n")
gold

In [ ]:
from sklearn.metrics import classification_report
result = classification_report(gold,pred,output_dict=True)
tweeteval_result = result['macro avg']['f1-score']
for k in result:
  print(k, result[k])
  print(f"{'-'*30}\nTweetEval Score (emotion): {tweeteval_result}")

In [ ]:
# download label mapping for stance
import csv
import urllib.request
task= 'stance'
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/stance/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [column[1] for column in csvreader if len(column) > 1]
labels

In [ ]:
!wget https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/stance/atheism/test_text.txt

In [ ]:
stance_text="test_text.txt"
ptext = open(stance_text, "r").read().split("\n")
ptext

In [ ]:
gold_2 = stext;
for k in range(len(stext)):
  text = preprocess_tweet(stext[k])
  encoded_input = tokenizer(text, return_tensors='tf')
  output = model(encoded_input)
  scores = output[0][0].numpy()
  scores = softmax(scores)
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    if i == 0:
        lb =l;  # to get first ranking number
    s = scores[ranking[i]]
    #print(f"{i+1}) {l} {np.round(float(s), 4)}")
    s=len(labels);
  for j in range(len(labels)):
      if lb == labels[j]:
         value = j;
  gold_2[k]=value;
gold_2

In [ ]:
!wget https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/predictions/hate.txt

In [ ]:
gold_2
for i in range(len(gold_2)):
   if gold_2[i]==0:
      gold_2[i]='0'
   if gold_2[i]==1:
      gold_2[i]='1'
   if gold_2[i]==2:
      gold_2[i]='2'

In [ ]:
# Read the stance.

example1 = "stance.txt"
pred = open(example1, "r").read().split("\n")
pred =gold_2

In [ ]:
!wget https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/stance/atheism/test_labels.txt

In [ ]:
test_labels = "test_labels.txt"
gold = open(test_labels, "r").read().split("\n")
gold

In [ ]:
from sklearn.metrics import classification_report
result = classification_report(gold,pred,output_dict=True)
tweeteval_result = result['macro avg']['f1-score']
for k in result:
  print(k, result[k])
  print(f"{'-'*30}\nTweetEval Score (emotion): {tweeteval_result}")